#### Implementing LangGraph's persistence layer for fault tolerance and facilitate conversations (remember previous interactions)

In [ ]:
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
import os 

load_dotenv()

# GPT_MODEL = 'gpt-4o'
model = ChatOpenAI(model="gpt-4o")

In [ ]:
# API Configuration
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.graph import START, MessagesState, StateGraph
from psycopg import Connection

In [44]:
# Database connection details
DB_URI = os.getenv("DB_URI")

connection_kwargs = {
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": "5432",
    "autocommit": True,
    "prepare_threshold": 0,
}

In [45]:
# Define LangGraph workflow
workflow = StateGraph(state_schema=MessagesState)

In [46]:
# Define function to call the model

def call_model(state: MessagesState):
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability."
    )
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = model.invoke(messages)
    return {"messages": response}

In [47]:
# Add the node and edge to the workflow

workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

In [55]:
with Connection.connect(DB_URI, **connection_kwargs) as conn:
    checkpointer = PostgresSaver(conn)

    # first time using database/table only
    # checkpointer.setup()

    # compile the workflow with the Postgres checkpointer
    app = workflow.compile(checkpointer=checkpointer)

    # Example configuration and input
    config = {"configurable": {"thread_id": "1"}}
    input_data = {"messages": [("human", "what's the weather in Denver?")]}

    # run the workflow
    response = app.invoke(input_data, config)

    # retrieve the checkpoint tuple
    checkpoint_tuple = checkpointer.get_tuple(config)

    print("Workflow Response:", response)
    print("Checkpoint Tuple:", checkpoint_tuple)

Workflow Response: {'messages': [HumanMessage(content="what's the weather in Denver?", additional_kwargs={}, response_metadata={}, id='ddaccb70-839b-4a6d-b686-f615e5e2f34e'), AIMessage(content="I'm unable to provide real-time weather updates. To get the current weather in Denver, I recommend checking a reliable weather website or app like the Weather Channel, the National Weather Service, or a weather app on your smartphone.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 34, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-a692fdca-00cd-4ed7-8075-6fff37822903-0', usage_metadata={'input_tokens': 34, 'output_tokens': 45, '

In [56]:
checkpoint_tuple

CheckpointTuple(config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efdbd30-0d9e-67bc-8004-b7cff6ffd96d'}}, checkpoint={'v': 1, 'id': '1efdbd30-0d9e-67bc-8004-b7cff6ffd96d', 'ts': '2025-01-26T10:47:57.063775+00:00', 'pending_sends': [], 'versions_seen': {'model': {'start:model': '00000000000000000000000000000005.0.7492994878569476'}, '__input__': {}, '__start__': {'__start__': '00000000000000000000000000000004.0.24078976381853234'}}, 'channel_versions': {'model': '00000000000000000000000000000006.0.9491617378314615', 'messages': '00000000000000000000000000000006.0.08219908600831227', '__start__': '00000000000000000000000000000005.0.11068437511918705', 'start:model': '00000000000000000000000000000006.0.6516098280634158'}, 'channel_values': {'model': 'model', 'messages': [HumanMessage(content="what's the weather in Denver?", additional_kwargs={}, response_metadata={}, id='ddaccb70-839b-4a6d-b686-f615e5e2f34e'), AIMessage(content="I'm unable to provide real-

In [66]:
# Test persistence

with Connection.connect(DB_URI, **connection_kwargs) as conn:
    checkpointer = PostgresSaver(conn)

    # first time using database/table only
    # checkpointer.setup()

    # compile the workflow with the Postgres checkpointer
    app = workflow.compile(checkpointer=checkpointer)

    config = {"configurable": {"thread_id": "1"}}
    input_data = {"messages": [("human", "What city did I ask you for weather information?")]}

    response = app.invoke(input_data, config)

    checkpoint_tuple = checkpointer.get_tuple(config)

    print("Workflow Response:", response)
    print("Checkpoint Tuple:", checkpoint_tuple)

Workflow Response: {'messages': [HumanMessage(content="what's the weather in Denver?", additional_kwargs={}, response_metadata={}, id='ddaccb70-839b-4a6d-b686-f615e5e2f34e'), AIMessage(content="I'm unable to provide real-time weather updates. To get the current weather in Denver, I recommend checking a reliable weather website or app like the Weather Channel, the National Weather Service, or a weather app on your smartphone.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 34, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-a692fdca-00cd-4ed7-8075-6fff37822903-0', usage_metadata={'input_tokens': 34, 'output_tokens': 45, '

In [67]:
response

{'messages': [HumanMessage(content="what's the weather in Denver?", additional_kwargs={}, response_metadata={}, id='ddaccb70-839b-4a6d-b686-f615e5e2f34e'),
  AIMessage(content="I'm unable to provide real-time weather updates. To get the current weather in Denver, I recommend checking a reliable weather website or app like the Weather Channel, the National Weather Service, or a weather app on your smartphone.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 34, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-a692fdca-00cd-4ed7-8075-6fff37822903-0', usage_metadata={'input_tokens': 34, 'output_tokens': 45, 'total_tokens': 79

In [54]:
# import psycopg

# try:
#     # Establish the connection
#     with psycopg.connect(DB_URI, **connection_kwargs) as conn:
#         print("Connection successful!")

#         # Execute a test query
#         with conn.cursor() as cur:
#             cur.execute("""
#                 SELECT table_name
#                 FROM information_schema.tables
#                 WHERE table_schema = 'public'
#                 ORDER BY table_name;
#             """)
#             tables = cur.fetchall()
            
#             # Print the list of tables
#             print("Tables in the database:")
#             for table in tables:
#                 print(f"- {table[0]}")

# except Exception as e:
#     print("Error connecting to PostgreSQL:", e)